In [ ]:
#Import and helper functions
import sqlite3
import pandas as pd
import re
import numpy as np 
import sys
import json

#Helper functions from Kolton
def get_col_names(cursor, table_name):
    '''
    Retrieves the column names for a table in a sqlite3 db.
    ------
    cursor: sqliteConnection cursor object
    table_name: table_name to get column names for
    '''
    cursor.execute(f"""
    SELECT sql FROM sqlite_master WHERE name='{table_name}';
    """)
    
    res = mimiciii.fetchall()
    cols = re.findall(r'\"\w+\"', res[0][0])
    return [x[1:-1] for x in cols]

def get_df_from_table_from_db(cursor, table_name, num_rows='*', skip_cols=[]):
    '''
    Retreives table from sqlite3 db in form of df
    ------
    cursor: sqliteConnection cursor object
    table_name: name of table to get from cursor db
    num_rows: number of rows to retrieve (or '*' for all rows)
    skip_cols: list of columns to skip in the retrieval
    '''
    col_names = get_col_names(cursor, table_name)
    
    use_cols = [col for col in col_names if col not in skip_cols]
    
    if num_rows == '*':
        query = f'''select {', '.join(use_cols)} from {table_name};'''
    else:
        query = f'''select {', '.join(use_cols)} from {table_name} limit {num_rows};'''
        
    cursor.execute(query)
    rows = cursor.fetchall()
    
    return pd.DataFrame(rows, columns=use_cols)

def get_tables_list_from_db(cursor):
    cursor.execute("""
    select name from sqlite-master where type='table';
    """)
    table_names = [table[0] for table in cursor.fetchall()]
    return table_names

def to_int(x):
    if not x or np.isnan(x):
        return 0
    return int(x)

In [ ]:
#relative path to where the mimic3.db file is
db_path = 'D:/mimic-iii-clinical-database-1.4/mimic3.db'
#connection object to db
sqliteConnection = sqlite3.connect(db_path)
#cursor/pointer
mimiciii = sqliteConnection.cursor()

In [ ]:
#Extract admission table with designated attributes
admission = get_df_from_table_from_db(mimiciii, 'admissions')
admission = admission.apply(lambda x: x.astype(str).str.upper())
admission.HADM_ID = admission.HADM_ID.astype('int64')
admission.SUBJECT_ID = admission.SUBJECT_ID.astype('int64')

admission = admission[['HADM_ID','SUBJECT_ID','DIAGNOSIS']]
admission.head()

In [ ]:
#Import json for HADM_ID list of arf diagnosis
arf_data = []
with open('arf_hadm_ids.json') as json_file:
   arf_data = json.load(json_file)
admission.loc[admission['HADM_ID'].isin(list(map(int, arf_data)))]

In [ ]:
#Mark arf as arf, and the rest as other
admission['WR_DIAGNOSIS'] = np.full(len(admission),"Other")
admission.loc[admission['HADM_ID'].isin(list(map(int, arf_data))),'WR_DIAGNOSIS'] = "ARF" 
admission['WR_DIAGNOSIS'].value_counts()

In [ ]:
# Read the labevents table, which is huge
labevents = get_df_from_table_from_db(mimiciii, 'labevents')
labevents = labevents[~np.isnan(labevents['HADM_ID'])]
labevents.HADM_ID = labevents.HADM_ID.astype('int64')
labevents.to_csv("temp_labevent.csv",index=False)
labevents.head()

In [ ]:
# Read the labitems table for ITEMID-test name mapping
test_names = get_df_from_table_from_db(mimiciii, 'd_labitems')
test_names.head()

In [ ]:
# There are too many tests types and not all of them are performed for a HADM_ID
# So instead, we pick the most common 10 tests to decrease the number of dropped tests when doing dropna 
most_common_tests = labevents.groupby('ITEMID').count().sort_values(by='HADM_ID', ascending=False)[0:10]
most_common_tests = pd.DataFrame({'ITEMID':most_common_tests.index,'COUNT':most_common_tests['HADM_ID']})
most_common_tests.reset_index(drop=True,inplace=True)
most_common_tests = most_common_tests.merge(test_names,on='ITEMID',how='left')[['ITEMID','LABEL']]
most_common_tests

In [ ]:
# Filter the labevents according to the most common tests
labevents = labevents[labevents['ITEMID'].isin(most_common_tests['ITEMID'].unique())]
labevents

In [ ]:
# Align two tables
labevents = labevents[labevents['HADM_ID'].isin(admission['HADM_ID'].unique())]
admission = admission[admission['HADM_ID'].isin(labevents['HADM_ID'].unique())]
print(len(labevents['HADM_ID'].unique()))
print(len(admission['HADM_ID'].unique()))

In [ ]:
# Make columns for most common tests' values and abnormalities
import warnings
warnings.filterwarnings("ignore")
for name in most_common_tests['LABEL']:
    admission.loc[:,name] = np.full(len(admission),"")
    admission.loc[:,name+"_ab"] = np.full(len(admission),False)
admission

In [ ]:
# Collect values form labevents and fill them into admission dataframe
for id in admission['HADM_ID']:
    temp = labevents[labevents['HADM_ID'] == id]
    for item in most_common_tests['ITEMID']:
        row = admission['HADM_ID'] == id
        col = most_common_tests[most_common_tests['ITEMID'] == item]['LABEL'].values[0]
        if len(temp[temp['ITEMID'] == item]['VALUE'].values) <= 0:
            admission.loc[row, col] = np.NaN
            admission.loc[row, col+'_ab'] = np.NaN
        else:
            admission.loc[row, col] = temp[temp['ITEMID'] == item]['VALUE'].values[0]
            admission.loc[row, col+'_ab'] = temp[temp['ITEMID'] == item]['FLAG'].values[0] == "abnormal"

In [ ]:
# Save the data for later use
admission.to_csv("ED_Wrangling_Result.csv",index=False)